In [26]:
import numpy as np
from pde_find import PDEFind
from IPython.display import display, Math

In [27]:
raw_data = np.load("data/3.npz")

u = raw_data["u"]
v = raw_data["u"]
x = raw_data["x"]
y = raw_data["y"]
t = raw_data["t"]

data = np.stack([x, y, t, u, v], axis=-1)

# downsample the data
data = data[::4, ::4, ::4]

vars = (["x", "y", "t"], ["u", "v"])

In [28]:
pdefind = PDEFind(data, vars, lib_size=3, order=2)

In [29]:
library, labels = pdefind.create_library()

print("len(library):", len(library))
print("len(library):", len(set(labels)))

# for i in range(len(library)):
#     display(Math(labels[i]))

len(library): 455
len(library): 455


In [30]:
u_t = np.gradient(data[..., 3], data[..., 2][0, 0], axis=-1)
v_t = np.gradient(data[..., 4], data[..., 2][0, 0], axis=-1)

In [36]:
print("Solving for u")

coef_u, alpha_u = pdefind.solve_regression(library, u_t, algorithm="lasso")

display(Math(pdefind.latex_string(coef_u, "u")))

Solving for u
Solving using lasso regression
Library size: 95047680, Target size: 208896
# of non-zero terms: 48


<IPython.core.display.Math object>

In [32]:
print("Solving for u")

coef_u_ridge, alpha_u_ridge = pdefind.solve_regression(library, u_t, algorithm="ridge", alphas=[10e3])
print("alpah:", alpha_u_ridge)

display(Math(pdefind.latex_string(coef_u_ridge, "u")))

Solving for u
Solving using ridge regression
Library size: 95047680, Target size: 208896
# of non-zero terms: 113
alpah: 10000.0


<IPython.core.display.Math object>

In [35]:
print("Solving for v")

coef_v, alpha_v = pdefind.solve_regression(library, v_t, algorithm="tlsq", cutoff=10)

display(Math(pdefind.latex_string(coef_v, "v")))

Solving for v
Solving using tlsq regression
Library size: 95047680, Target size: 208896


TLSQ Iterations:   6%|▌         | 3/50 [00:07<02:04,  2.65s/it, # of non-zero terms=280]


KeyboardInterrupt: 

In [ ]:
included_terms_u = pdefind.non_zero_terms(coef_u)
included_terms_v = pdefind.non_zero_terms(coef_v)
print("Included terms for u:")
display(Math((",\\,".join(included_terms_u))))
print("Included terms for v:")
display(Math((",\\,".join(included_terms_v))))